# PART 1 OF CODING - RETRIEVAL OF LIST OF FAST FOOD RESTAURANTS IN EVERY NEIGHBORHOOD OF NEW YORK

In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import display, HTML
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.0.2p             |    h14c3975_1002         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    conda-4.6.0                |        py36_1000         878 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0         conda-forge
    geopy:         1.18.1-py_0       conda-forge

The following packages will be UPDATED:

    conda:         4.5.12-py36_1000  conda-forge --> 4.6.0-py36_1000      conda-forge
    o

## Parsing the ZipCodes and Demographic csv file into a dataframe

In [3]:
df_data_1 = pd.read_csv('ny_zip_code_database.csv')       # Data Source - https://www.unitedstateszipcodes.org/ny/
df_data_2 = pd.read_csv('ny_data.csv')                    # Data Source - https://zipwho.com/
print(df_data_1.shape)
display(HTML(df_data_1.head().to_html()))
print(df_data_2.shape)
display(HTML(df_data_2.head().to_html()))


(42632, 15)


,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


(25, 37)


,zip,city,state,MedianIncome,MedianIncomeRank,CostOfLivingIndex,CostOfLivingRank,MedianMortgageToIncomeRatio,MedianMortgageToIncomeRank,OwnerOccupiedHomesPercent,OwnerOccupiedHomesRank,MedianRoomsInHome,MedianRoomsInHomeRank,CollegeDegreePercent,CollegeDegreeRank,ProfessionalPercent,ProfessionalRank,Population,PopulationRank,AverageHouseholdSize,AverageHouseholdSizeRank,MedianAge,MedianAgeRank,MaleToFemaleRatio,MaleToFemaleRank,MarriedPercent,MarriedRank,DivorcedPercent,DivorcedRank,WhitePercent,WhiteRank,BlackPercent,BlackRank,AsianPercent,AsianRank,HispanicEthnicityPercent,HispanicEthnicityRank
0,10001,New York,NY,40932,56,129.9,67,50.1,99,21.5,1,2.7,0,50.1,94,55.6,96,17007,70,1.9,1,36.4,42,100.2,83,33.1,1,9.2,46,54.2,14,7.9,73,14.3,97,19.8,89
1,10003,New York,NY,60891,87,371.7,97,19.1,19,26.5,2,2.7,0,70.1,99,66.2,99,54084,98,1.8,1,33.7,20,100.4,84,29.6,0,7.6,23,72.9,27,4.0,64,11.8,97,7.4,77
2,10010,New York,NY,62467,89,288.6,95,17.5,4,24.0,1,2.8,0,71.2,99,66.7,99,26471,82,1.7,0,37.6,56,83.5,11,35.2,2,8.6,37,73.0,27,5.7,69,10.1,96,9.3,80
3,10011,New York,NY,61986,88,347.1,96,27.3,96,30.6,2,2.7,0,68.5,99,69.1,99,46694,96,1.6,0,38.5,65,111.2,95,29.5,0,9.3,48,74.6,28,4.4,65,5.8,92,12.2,84
4,10012,New York,NY,58313,85,691.8,99,24.3,86,17.3,1,2.7,0,64.6,98,65.7,99,26031,82,1.8,0,34.6,25,108.0,94,35.7,2,7.0,16,68.6,23,3.0,60,17.4,98,7.2,77


## Filtering zipcode file for New York county

In [4]:
df_data_1=df_data_1[df_data_1['state'].str.contains('NY')]
df_data_1.dropna(subset = ['county'],inplace=True)
#print(df_data_1.county.unique())
df_data_1=df_data_1[df_data_1['county'].str.contains('New York')].reset_index()
print(df_data_1.shape)
display(HTML(df_data_1.head().to_html()))

(164, 16)


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,index,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,3699,10001,STANDARD,0,New York,NaN,"Empire State, G P O, Greeley Square, Macys Fin...",NY,New York County,America/New_York,"718,917,347,646",NaN,US,40.75,-74.00,19710
1,3700,10002,STANDARD,0,New York,Knickerbocker,"Manhattan, New York City, Ny, Ny City, Nyc",NY,New York County,America/New_York,718,NaN,US,40.71,-73.99,71450
2,3701,10003,STANDARD,0,New York,NaN,"Cooper, Manhattan",NY,New York County,America/New_York,"212,347,646,718,917",NaN,US,40.73,-73.99,39470
3,3702,10004,STANDARD,0,New York,Bowling Green,NaN,NY,New York County,America/New_York,"212,347,646,718,917",NaN,US,40.69,-74.02,4380
4,3703,10005,STANDARD,0,New York,Wall Street,"Manhattan, Nyc",NY,New York County,America/New_York,"347,718,212,646,917",NaN,US,40.71,-74.01,8600


[10001 10002 10003 10004 10005 10006 10007 10008 10009 10010 10011 10012
 10013 10014 10015 10016 10017 10018 10019 10020 10021 10022 10023 10024
 10025 10026 10027 10028 10029 10030 10031 10032 10033 10034 10035 10036
 10037 10038 10039 10040 10041 10043 10044 10045 10046 10047 10048 10055
 10060 10065 10069 10072 10075 10079 10080 10081 10082 10087 10090 10094
 10095 10096 10098 10099 10101 10102 10103 10104 10105 10106 10107 10108
 10109 10110 10111 10112 10113 10114 10115 10116 10117 10118 10119 10120
 10121 10122 10123 10124 10125 10126 10128 10129 10130 10131 10132 10133
 10138 10149 10150 10151 10152 10153 10154 10155 10156 10157 10158 10159
 10160 10161 10162 10163 10164 10165 10166 10167 10168 10169 10170 10171
 10172 10173 10174 10175 10176 10177 10178 10179 10184 10185 10196 10197
 10199 10203 10211 10212 10213 10242 10249 10256 10257 10258 10259 10260
 10261 10265 10268 10269 10270 10271 10272 10273 10274 10275 10276 10277
 10278 10279 10280 10281 10282 10285 10286 10292]


## Merging information from ZipCode and Demographic files so as to focus on relevant areas

In [30]:
df_relevant=pd.merge(df_data_2,df_data_1,left_on='zip',right_on='zip',how='left').reset_index()
df_relevant=df_relevant[['zip','latitude','longitude','MedianIncome','CostOfLivingIndex','Population','MedianAge']]
print(df_relevant.shape)
display(HTML(df_relevant.head().to_html()))
print(df_relevant.zip.nunique())

(25, 7)


,zip,latitude,longitude,MedianIncome,CostOfLivingIndex,Population,MedianAge
0,10001,40.75,-74.00,40932,129.9,17007,36.4
1,10003,40.73,-73.99,60891,371.7,54084,33.7
2,10010,40.74,-73.98,62467,288.6,26471,37.6
3,10011,40.74,-74.00,61986,347.1,46694,38.5
4,10012,40.73,-74.00,58313,691.8,26031,34.6


25


## Picturing zipcodes on New York map

In [9]:
address = 'New York'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(latitude, longitude))
map_ny = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, zipCd in zip(df_relevant['latitude'], df_relevant['longitude'], df_relevant['zip']):
    label = '{}'.format(zipCd)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of New York are 40.7308619, -73.9871558.


## Setting up Foursquare Credentials

In [10]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


## Looking at first zipcode on New York county

In [13]:
neighbourhood_latitude = df_relevant.loc[0, 'latitude'] # neighbourhood latitude value
neighbourhood_longitude = df_relevant.loc[0, 'longitude'] # neighbourhood longitude value

neighbourhood_name = df_relevant.loc[0, 'zip'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of 10001 are 40.75, -74.0.


## Retrieving list of fast food restaurants of first zipcode

In [ ]:
 LIMIT = 10 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CategoryId='4bf58dd8d48988d16e941735'
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
#url = 'https://api.foursquare.com/v2/venues/search?&client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT,
    CategoryId
)
print(url)
results = requests.get(url).json()
#results

## Defining a function so as to get list of fast food restaurants for each zipcode of New York

In [ ]:
def getNearbyVenues(zips, latitudes, longitudes, radius=500):
    
    venues_list=[]
    CategoryId='4bf58dd8d48988d16e941735'   # Category for fast food restaurant
    LIMIT = 10 # limit of number of venues returned by Foursquare API
    radius = 500 # define radius
    for zipCd, lat, lng in zip(zips, latitudes, longitudes):
        #print(zipCd)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT,
            CategoryId
)
        print(url)    
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            zipCd, 
            v['location']['lat'], 
            v['location']['lng'], 
            v['name'], 
            v['id'],
            v['categories'][0]['name']
        ) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['zip', 
                  'Latitude', 
                  'Longitude',
                  'Name',
                  'Id',
                  'Venue Category']
    print('Data Extracted!!')
    return(nearby_venues)

## Calling the function

In [16]:
ny_ff = getNearbyVenues(zips=df_relevant['zip'],
                                   latitudes=df_relevant['latitude'],
                                   longitudes=df_relevant['longitude']
                                  )


print('Data saved and exported!!')

Data Extracted!!
Data saved and exported!!


## Seeing the result's shape and structure

In [29]:
print(ny_ff.shape)
display(HTML(ny_ff.head().to_html()))
display(HTML(ny_ff.groupby('zip').count().to_html()))
print(ny_ff.zip.nunique())


(173, 6)


,zip,Latitude,Longitude,Name,Id,Venue Category
0,10001,40.748724,-74.003422,SUBWAY,4ecbe682f9f4a82587658269,Sandwich Place
1,10001,40.747569,-73.997088,McDonald's,4b1331eef964a5205c9523e3,Fast Food Restaurant
2,10001,40.752250,-74.005727,Enfes,59c143aa6cf01a3c2418005a,Turkish Restaurant
3,10001,40.748357,-74.004128,Subway,5a64e13cea1e445448e69273,Sandwich Place
4,10001,40.746259,-73.997676,Subway,4d6e842538363704f4d8a9d0,Sandwich Place


,Latitude,Longitude,Name,Id,Venue Category
zip,,,,,
10001,10,10,10,10,10
10003,10,10,10,10,10
10010,10,10,10,10,10
10011,10,10,10,10,10
10012,8,8,8,8,8
10013,10,10,10,10,10
10014,1,1,1,1,1
10016,10,10,10,10,10
10017,9,9,9,9,9


23


## Dropping duplicate restaurant information, if any

In [35]:
ny_ff.drop_duplicates(subset=['Id'], keep="first",inplace=True)
ny_ff.reset_index()
print(ny_ff.shape)
print(ny_ff.zip.nunique())
display(HTML(ny_ff.loc[ny_ff['zip'] == 10001].to_html()))

(173, 6)
23


,zip,Latitude,Longitude,Name,Id,Venue Category
0,10001,40.748724,-74.003422,SUBWAY,4ecbe682f9f4a82587658269,Sandwich Place
1,10001,40.747569,-73.997088,McDonald's,4b1331eef964a5205c9523e3,Fast Food Restaurant
2,10001,40.752250,-74.005727,Enfes,59c143aa6cf01a3c2418005a,Turkish Restaurant
3,10001,40.748357,-74.004128,Subway,5a64e13cea1e445448e69273,Sandwich Place
4,10001,40.746259,-73.997676,Subway,4d6e842538363704f4d8a9d0,Sandwich Place
5,10001,40.751434,-73.993487,Subway,4aef3eb9f964a520e3d621e3,Sandwich Place
6,10001,40.749272,-73.995269,Taco Bell / Pizza Hut,4f18c538e4b09594f839d493,Fast Food Restaurant
7,10001,40.753343,-73.995942,Subway,4ddedbc3cc3f89c0826fa9c8,Sandwich Place
8,10001,40.754320,-73.998480,Subway Restaurant,4b64bfb7f964a52016cd2ae3,Sandwich Place
9,10001,40.745323,-73.997815,Boston Market,3fd66200f964a52065e91ee3,American Restaurant


## Due to limited Foursquare regular calls, saving this information in csv file 'export_dataframe1.csv'

In [19]:
import base64
from IPython.display import HTML

def create_download_link( ny_ff, title = "Download CSV file", filename = "export_dataframe1.csv"):  
    csv = ny_ff.to_csv(index = None, header=True)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(ny_ff)